<a href="https://colab.research.google.com/github/afortuny/DeepLearningFastAI/blob/main/13_language_model_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 14.9 MB/s 
     |████████████████████████████████| 4.7 MB 24.6 MB/s 
     |████████████████████████████████| 365 kB 45.5 MB/s 
     |████████████████████████████████| 1.3 MB 50.0 MB/s 
     |████████████████████████████████| 120 kB 75.9 MB/s 
     |████████████████████████████████| 212 kB 76.0 MB/s 
     |████████████████████████████████| 115 kB 70.9 MB/s 
     |████████████████████████████████| 127 kB 76.3 MB/s 
     |████████████████████████████████| 6.6 MB 50.0 MB/s 
Mounted at /content/gdrive


In [3]:
#hide
from fastbook import *
from IPython.display import display,HTML

# Music model from scratch

In [8]:
!pip install music21
!apt-get install -y lilypond

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-droid-fallback fonts-lmodern fonts-noto-mono ghostscript gsfonts
  libauthen-sasl-perl libcupsfilters1 libcupsimage2 libdata-dump-perl
  libencode-locale-perl libfile-listing-perl libfont-afm-perl libgs9
  libgs9-common libhtml-form-perl libhtml-format-perl libhtml-parser-perl
  libhtml-tagset-perl libhtml-tree-perl libhttp-cookies-perl
  libhttp-daemon-perl libhttp-date-perl libhttp-message-perl
  libhttp-negotiate-perl libijs-0.35 libio-html-perl libio-socket-ssl-perl
  libjbig2dec0 libkpathsea6 liblwp-mediatypes-perl liblwp-protocol-https-perl
  libmailtools-perl libnet-http-perl libnet-smtp-ssl-per

In [5]:
#Importing Libraries
import numpy as np 
import pandas as pd 
from collections import Counter
import random
import IPython
from IPython.display import Image, Audio
import music21
from music21 import *
import matplotlib.pyplot as plt 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline
import sys
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter("ignore")
np.random.seed(42)

In [12]:
#Loading the list of chopin's midi files as stream 
filepath = "/content/gdrive/MyDrive/Music/bach"
#Getting midi files
all_midis= []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath+'/'+i
        midi = converter.parse(tr)
        all_midis.append(midi)

In [13]:
all_midis

[<music21.stream.Score 0x7fdf672e2890>,
 <music21.stream.Score 0x7fdf668b8810>,
 <music21.stream.Score 0x7fdf67b5a3d0>]

In [14]:
#Helping function        
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)
print("Total notes in all the Chopin midis in the dataset:", len(Corpus))

Total notes in all the Chopin midis in the dataset: 4586


In [15]:
Corpus

['C5',
 'C3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'C4',
 'E-3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'C5',
 'C3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'C4',
 'E-3',
 'E-4',
 'G3',
 'D4',
 'F3',
 'E-4',
 'G3',
 'G#4',
 'C3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'C4',
 'F3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'G#4',
 'C3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'C4',
 'F3',
 'F4',
 'G#3',
 'E4',
 'G3',
 'F4',
 'G#3',
 'B4',
 'C3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'D4',
 'F3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'B4',
 'C3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'D4',
 'F3',
 'F4',
 'G#3',
 'E-4',
 'G3',
 'F4',
 'G#3',
 'C5',
 'C3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'E-4',
 'G3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'C5',
 'C3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'E-4',
 'G3',
 'G4',
 'E-3',
 'F4',
 'D3',
 'G4',
 'E-3',
 'E-5',
 'C3',
 'G#4',
 'C4',
 'G4',
 'B-3',
 'G#4',
 'C4',
 '

In [16]:
from fastai.text.all import *

In [17]:
text = ' . '.join([l.strip() for l in Corpus])
text[:100]

'C5 . C3 . E-4 . G3 . D4 . F3 . E-4 . G3 . C4 . E-3 . E-4 . G3 . D4 . F3 . E-4 . G3 . C5 . C3 . E-4 .'

In [18]:
tokens = text.split(' ')
tokens[:10]

['C5', '.', 'C3', '.', 'E-4', '.', 'G3', '.', 'D4', '.']

In [19]:
vocab = L(*tokens).unique()

In [22]:
word2idx = {w:i for i,w in enumerate(vocab)}
nums = L(word2idx[i] for i in tokens)
nums

(#9171) [0,1,2,1,3,1,4,1,5,1...]

In [23]:
L((tokens[i:i+3], tokens[i+3]) for i in range(0,len(tokens)-4,3))

(#3056) [(['C5', '.', 'C3'], '.'),(['.', 'E-4', '.'], 'G3'),(['G3', '.', 'D4'], '.'),(['.', 'F3', '.'], 'E-4'),(['E-4', '.', 'G3'], '.'),(['.', 'C4', '.'], 'E-3'),(['E-3', '.', 'E-4'], '.'),(['.', 'G3', '.'], 'D4'),(['D4', '.', 'F3'], '.'),(['.', 'E-4', '.'], 'G3')...]

In [24]:
seqs = L((tensor(nums[i:i+3]), nums[i+3]) for i in range(0,len(nums)-4,3))
seqs

(#3056) [(tensor([0, 1, 2]), 1),(tensor([1, 3, 1]), 4),(tensor([4, 1, 5]), 1),(tensor([1, 6, 1]), 3),(tensor([3, 1, 4]), 1),(tensor([1, 7, 1]), 8),(tensor([8, 1, 3]), 1),(tensor([1, 4, 1]), 5),(tensor([5, 1, 6]), 1),(tensor([1, 3, 1]), 4)...]

In [ ]:
bs = 64
cut = int(len(seqs) * 0.8)
dls = DataLoaders.from_dsets(seqs[:cut], seqs[cut:], bs=64, shuffle=False)